In [ ]:
#Decision Tree
clf_entropy = DecisionTreeClassifier(criterion = "entropy", max_depth = 3)
clf_entropy.fit(X_train, Y_train)
plt.figure(figsize=(20,15))
tree.plot_tree(clf_entropy, feature_names=data.iloc[0], filled=True)\
y_pred = clf_object.predict(X_test)
y_pred_entropy = prediction(X_test, clf_entropy)

#KNN
X_train, X_test = standardscaler(X_train, X_test)
k_values = list(range(1, 20))
scores = []
for k in k_values:
   knn = KNeighborsClassifier(n_neighbors=k)
   score = cross_val_score(knn, X, Y, cv=3)
   scores.append(np.mean(score))
best_k = k_values[np.argmax(scores)]
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, Y_train)
y_pred = best_knn.predict(X_test)
sb.lineplot(x=k_values, y=scores, marker='o')

#NB
NBclassifier = GaussianNB()
NBmodel = NBclassifier.fit(train, train_labels)
NBpreds = NBclassifier.predict(test)
print("PREDICTIONS ARE AS FOLLOWS :\n", NBpreds[:30])
print("\n ACCURACY : ", accuracy_score(test_labels, NBpred)*100)

#KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy.random import uniform
from sklearn.datasets import make_blobs
import seaborn as sns
import random
def euclidean(point, data):
    """
    Euclidean distance between point & data.
    Point has dimensions (m,), data has dimensions (n,m), and output will be of size (n,).
    """
    return np.sqrt(np.sum((point - data)**2, axis=1))
class KMeans:
    def __init__(self, n_clusters=8, max_iter=300):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
    def fit(self, X_train):
        # Initialize the centroids, using the "k-means++" method, where a random datapoint is selected as the first,
        # then the rest are initialized w/ probabilities proportional to their distances to the first
        # Pick a random point from train data for first centroid
        self.centroids = [random.choice(X_train)]
        for _ in range(self.n_clusters-1):
            # Calculate distances from points to the centroids
            dists = np.sum([euclidean(centroid, X_train) for centroid in self.centroids], axis=0)
            # Normalize the distances
            dists /= np.sum(dists)
            # Choose remaining points based on their distances
            new_centroid_idx, = np.random.choice(range(len(X_train)), size=1, p=dists)
            self.centroids += [X_train[new_centroid_idx]]
        # This initial method of randomly selecting centroid starts is less effective
        # min_, max_ = np.min(X_train, axis=0), np.max(X_train, axis=0)
        # self.centroids = [uniform(min_, max_) for _ in range(self.n_clusters)]
        # Iterate, adjusting centroids until converged or until passed max_iter
        iteration = 0
        prev_centroids = None
        while np.not_equal(self.centroids, prev_centroids).any() and iteration < self.max_iter:
            # Sort each datapoint, assigning to nearest centroid
            sorted_points = [[] for _ in range(self.n_clusters)]
            for x in X_train:
                dists = euclidean(x, self.centroids)
                centroid_idx = np.argmin(dists)
                sorted_points[centroid_idx].append(x)
            # Push current centroids to previous, reassign centroids as mean of the points belonging to them
            prev_centroids = self.centroids
            self.centroids = [np.mean(cluster, axis=0) for cluster in sorted_points]
            for i, centroid in enumerate(self.centroids):
                if np.isnan(centroid).any():  # Catch any np.nans, resulting from a centroid having no points
                    self.centroids[i] = prev_centroids[i]
            iteration += 1
    def evaluate(self, X):
        centroids = []
        centroid_idxs = []
        for x in X:
            dists = euclidean(x, self.centroids)
            centroid_idx = np.argmin(dists)
            centroids.append(self.centroids[centroid_idx])
            centroid_idxs.append(centroid_idx)
        return centroids, centroid_idxs
# Create a dataset of 2D distributions
centers = 5
X_train, true_labels = make_blobs(n_samples=100, centers=centers, random_state=42)
X_train = StandardScaler().fit_transform(X_train)
# Fit centroids to dataset
kmeans = KMeans(n_clusters=centers)
kmeans.fit(X_train)
# View results
class_centers, classification = kmeans.evaluate(X_train)
sns.scatterplot(x=[X[0] for X in X_train],
                y=[X[1] for X in X_train],
                hue=true_labels,
                style=classification,
                palette="deep",
                legend=None
                )
plt.plot([x for x, _ in kmeans.centroids],
         [y for _, y in kmeans.centroids],
         'k+',
         markersize=10,
         )
plt.show()



#Reinforcement
GRID_SIZE = 5
START_STATE = (0, 0)
GOAL_STATE = (GRID_SIZE - 1, GRID_SIZE - 1)
OBSTACLES = [(1, 1), (2, 2), (3, 3)]
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EPSILON = 0.1
NUM_EPISODES = 1000
q_values = np.zeros((GRID_SIZE, GRID_SIZE, 4))
def choose_action(state):
 if np.random.rand() < EPSILON:
 return np.random.choice(4)  # Explore
 else:
 return np.argmax(q_values[state])
def q_learning():
 for episode in range(NUM_EPISODES):
        state = START_STATE
 	while state != GOAL_STATE:
            action = choose_action(state)
            next_state = update_state(state, action)
            reward = get_reward(next_state)
            q_values[state][action] = (1 - LEARNING_RATE) * q_values[state][action] + \
         					LEARNING_RATE * (reward + DISCOUNT_FACTOR * np.max
            state = next_state
def update_state(state, action):
    x, y = state
    if action == 0:  # Up
        x = max(0, x - 1)
    elif action == 1:  # Down
        x = min(GRID_SIZE - 1, x + 1)
    elif action == 2:  # Left
        y = max(0, y - 1)
    elif action == 3:  # Right
        y = min(GRID_SIZE - 1, y + 1)
 return (x, y)
def get_reward(state):
 	return -1 if state in OBSTACLES else 0 if state != GOAL_STATE else 1
q_learning()
 # Visualize the learned policy
def visualize_policy():
    policy = np.argmax(q_values, axis=2)
    plt.imshow(policy, cmap='cool', origin='upper')
    plt.title('Learned Policy for Grid World Navigation')
    plt.show()
 visualize_policy()

#3 Tensorflow
graph = tf.Graph()
 with graph.as_default():
  tensor_a = tf.constant(3.0)
  tensor_b = tf.constant(4.2)
  result = tf.add(tensor_a, tensor_b)
 print(graph)

#4 Regression
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(1)])
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train_scaled, y_train, epochs=100)
y_pred = model.predict(X_test_scaled)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')

#4 Kmeans
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_scaled)
y_kmeans = kmeans.predict(X_scaled)

#4 KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_knn_pred = knn.predict(X_test_scaled)
accuracy_knn = accuracy_score(y_test, y_knn_pred)*100
conf_matrix = confusion_matrix(y_test, y_knn_pred)

#5 Ensemble
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
random_forest = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)

#6 MLP
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
model = models.Sequential()
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',
 			rics=['accuracy'])
history=model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_data=(test_images, tes
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc * 100}%')
training_results = history
train_loss = training_results.history['loss']
train_acc = training_results.history['accuracy']
valid_loss = training_results.history['val_loss']
valid_acc = training_results.history['val_accuracy']
plt.plot(train_loss, label="Training Loss", color="g")
plt.plot(valid_loss, label="Validation Loss", color="b")
plt.plot(train_acc, label="Training Accuracy", color="g")
plt.plot(valid_acc, label="Validation Accuracy", color="b")